In [12]:
import sys,os,os.path
sys.path.insert(0,"./Library/Python/3.9/lib/python/site-packages")
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from datetime import datetime
import math

In [14]:
os.environ["MYSQL_HOST"]=""
os.environ["MYSQL_PORT"]=""
os.environ["MYSQL_USER"]=""
os.environ["MYSQL_PASSWORD"]=""

In [15]:
## mysql connection
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')

con = pymysql.connect(
      host=host,
      port=int(port),
      user=user,
      passwd=password,
      db='',
      charset='utf8mb4')

In [16]:
from sqlalchemy import create_engine
import pickle

EDS_LOGIN = 'livongo_ds'
EDS_PASSWD = 'ari1stoplhanesd'
EDS_HOST = 'datasunrise01.aws2.teladoc.com'
EDS_PORT = '3309'

engine = create_engine('mysql+pymysql://' + EDS_LOGIN + ':' + EDS_PASSWD + '@' + EDS_HOST + ':' + EDS_PORT + '/DW')

# set the campaign keys and send dates here
campaign_keys = [1412]
campaign_send_dt = '2020-08-25'

In [5]:

sql_inclusion = """
# Staged/Hybrid Primary Members Registrations
SELECT dm.`member_id`,a.eligible_member_key, a.final_receive_dt, dm.gender_cd, dm.dob_dt, dm.home_postal, a.client_segment_nm, a.group_nm, 'STAGED' as eligibility_type 
FROM 
(SELECT DISTINCT
ci.`member_durable_key`,
ci.eligible_member_key,
dc.final_receive_dt AS final_receive_dt,
org.client_segment_nm,
    org.group_nm
FROM 
DW.`fact_member_campaign_inclusion` ci 
 INNER JOIN DW.`dim_campaign` dc ON ci.campaign_key = dc.`campaign_key` AND ci.`campaign_key` in ({0})
 INNER JOIN DW.`dim_organization` org ON ci.org_key = org.org_key
WHERE ci.`member_durable_key` <> -1) a 
INNER JOIN DW.`dim_member` dm ON a.member_durable_key = dm.member_durable_key AND dm.scd_current_flg = 'Y' 
UNION ALL
# RTE Primary member Registrations
SELECT dem.registered_member_id as member_id, cii.eligible_member_key, cii.final_receive_dt, dem.gender_cd, dem.dob_dt, dem.home_postal, cii.client_segment_nm, cii.group_nm, 'RTE' as eligibility_type
FROM 
(
SELECT DISTINCT
ci.`member_durable_key`,
ci.eligible_member_key, 
dc.final_receive_dt AS final_receive_dt,
org.client_segment_nm,
    org.group_nm
FROM 
DW.`fact_member_campaign_inclusion` ci 
 INNER JOIN DW.`dim_campaign` dc ON ci.campaign_key = dc.`campaign_key` AND ci.`campaign_key` in ({0})
 INNER JOIN DW.`dim_organization` org ON ci.org_key = org.org_key
WHERE ci.`member_durable_key` = -1) cii  
INNER JOIN DW.dim_eligible_member dem ON cii.eligible_member_key = dem.eligible_member_key 
ORDER BY 1,2,3;
""".format(','.join([str(c) for c in campaign_keys]))

df_campaign_inclusions = pd.read_sql(
    sql_inclusion,
    engine.connect(),
    index_col=None,
    coerce_float=True,
    params=None,
    parse_dates=None,
    columns=None,
    chunksize=None)

pickle.dump(df_campaign_inclusions, open('df_campaign_inclusions.pickle','wb'))


In [11]:
# previous campaign inclusions for staged members
sql_prev_campaigns_staged = """
SELECT dm.`member_id`, count(*) as prev_campaign_inclusion_cnt
FROM 
DW.`fact_member_campaign_inclusion` ci 
 INNER JOIN DW.`dim_campaign` dc ON ci.campaign_key = dc.`campaign_key` AND dc.final_receive_dt < '{0}' and dc.campaign_mode_nm in ('TELADOC LETTER','TELADOC SELF MAILER')
 INNER JOIN DW.`dim_member` dm ON ci.member_durable_key = dm.member_durable_key AND dm.scd_current_flg = 'Y' 
WHERE ci.`member_durable_key` <> -1
group by dm.`member_id`
""".format(campaign_send_dt)

df_prev_campaigns_staged = pd.read_sql(
    sql_prev_campaigns_staged,
    engine.connect(),
    index_col=None,
    coerce_float=True,
    params=None,
    parse_dates=None,
    columns=None,
    chunksize=None)

pickle.dump(df_prev_campaigns_staged, open('df_prev_campaigns_staged.pickle','wb'))


OperationalError: (pymysql.err.OperationalError) (2013, 'Lost connection to MySQL server during query ([WinError 10054] An existing connection was forcibly closed by the remote host)')
[SQL: 
SELECT dm.`member_id`, count(*) as prev_campaign_inclusion_cnt
FROM 
DW.`fact_member_campaign_inclusion` ci 
 INNER JOIN DW.`dim_campaign` dc ON ci.campaign_key = dc.`campaign_key` AND dc.final_receive_dt < '2020-08-25' and dc.campaign_mode_nm in ('TELADOC LETTER','TELADOC SELF MAILER')
 INNER JOIN DW.`dim_member` dm ON ci.member_durable_key = dm.member_durable_key AND dm.scd_current_flg = 'Y' 
WHERE ci.`member_durable_key` <> -1
group by dm.`member_id`
]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
# Could not make it work through python so created a table through workbench and then loading it here
# CREATE TABLE livongo.derm21_1412_prev_campaigns_staged as
# SELECT dm.`member_id`, count(*) as prev_campaign_inclusion_cnt
# FROM 
# DW.`fact_member_campaign_inclusion` ci 
#  INNER JOIN DW.`dim_campaign` dc ON ci.campaign_key = dc.`campaign_key` AND dc.final_receive_dt < '2020-08-25' and dc.campaign_mode_nm in ('TELADOC LETTER','TELADOC SELF MAILER')
#  INNER JOIN DW.`dim_member` dm ON ci.member_durable_key = dm.member_durable_key AND dm.scd_current_flg = 'Y' 
# WHERE ci.`member_durable_key` <> -1
# group by dm.`member_id`;

In [19]:
# previous campaign inclusions for staged members
sql_prev_campaigns_staged = """
SELECT member_id, prev_campaign_inclusion_cnt
FROM livongo.derm21_1412_prev_campaigns_staged
"""

df_prev_campaigns_staged = pd.read_sql(
    sql_prev_campaigns_staged,
    engine.connect(),
    index_col=None,
    coerce_float=True,
    params=None,
    parse_dates=None,
    columns=None,
    chunksize=None)

pickle.dump(df_prev_campaigns_staged, open('df_prev_campaigns_staged.pickle','wb'))


In [17]:

# previous campaign inclusions for rte members
sql_prev_campaigns_rte = """
SELECT ci.eligible_member_key, count(*)
FROM 
DW.`fact_member_campaign_inclusion` ci 
 INNER JOIN DW.`dim_campaign` dc ON ci.campaign_key = dc.`campaign_key` and dc.final_receive_dt < '{0}' and dc.campaign_mode_nm in ('TELADOC LETTER','TELADOC SELF MAILER')
WHERE ci.`member_durable_key` = -1 and ci.eligible_member_key <> -1
group by ci.eligible_member_key
""".format(campaign_send_dt)

df_prev_campaigns_rte = pd.read_sql(
    sql_prev_campaigns_rte,
    engine.connect(),
    index_col=None,
    coerce_float=True,
    params=None,
    parse_dates=None,
    columns=None,
    chunksize=None)

pickle.dump(df_prev_campaigns_rte, open('df_prev_campaigns_rte.pickle','wb'))

In [20]:

# using staged members only
df_campaign_inclusions = df_campaign_inclusions.loc[df_campaign_inclusions['eligibility_type'] == 'STAGED'].merge(df_prev_campaigns_staged, how='left', on='member_id')
df_campaign_inclusions['prev_campaign_inclusion_cnt'].fillna(0, inplace=True)


In [21]:

# direct mail registrations
sql_registrations = """
# Staged/Hybrid Primary Members Registrations
SELECT dm.`member_id`,a.`Eligible Member Key`, DATE(dm.first_registration_dt) AS `Registration date`, a.final_receive_dt, a.`Eligibility Type`
FROM 
(SELECT DISTINCT
ci.`member_durable_key`,
ci.eligible_member_key AS `Eligible Member Key`
,dc.final_receive_dt AS final_receive_dt
,org.`primary_source_nm` AS `Eligibility Type`
FROM 
DW.`fact_member_campaign_inclusion` ci 
 INNER JOIN DW.`dim_campaign` dc ON ci.campaign_key = dc.`campaign_key` AND ci.`campaign_key` in ({0}) 
INNER JOIN DW.`dim_organization` org ON ci.org_key = org.org_key 
WHERE ci.`member_durable_key` <> -1) a 
INNER JOIN DW.`dim_member` dm ON a.member_durable_key = dm.member_durable_key AND dm.scd_current_flg = 'Y' 
WHERE  dm.first_registration_dt IS NOT NULL AND DATE(dm.first_registration_dt) >=  a.final_receive_dt
UNION ALL
# RTE Primary member Registrations
SELECT dm.`member_id`,cii.eligible_member_key, DATE(dm.first_registration_dt) AS `Registration date`, cii.final_receive_dt, cii.`Eligibility Type`
FROM 
(
SELECT DISTINCT
ci.`member_durable_key`,
ci.eligible_member_key 
,dc.final_receive_dt AS final_receive_dt
,org.`primary_source_nm` AS `Eligibility Type`
FROM 
DW.`fact_member_campaign_inclusion` ci 
 INNER JOIN DW.`dim_campaign` dc ON ci.campaign_key = dc.`campaign_key` AND ci.`campaign_key` in ({0})
INNER JOIN DW.`dim_organization` org ON ci.org_key = org.org_key 
WHERE ci.`member_durable_key` = -1) cii  
INNER JOIN DW.dim_eligible_member dem ON cii.eligible_member_key = dem.eligible_member_key 
INNER JOIN DW.`dim_member` dm ON dem.registered_member_id = dm.member_id  AND dm.first_registration_dt IS NOT NULL  AND dm.scd_current_flg = 'Y' 
WHERE dm.first_registration_dt >=  cii.final_receive_dt 
ORDER BY 1,2,3,4,5;
""".format(','.join([str(c) for c in campaign_keys]))

df_registrations = pd.read_sql(
    sql_registrations,
    engine.connect(),
    index_col=None,
    coerce_float=True,
    params=None,
    parse_dates=None,
    columns=None,
    chunksize=None)

pickle.dump(df_registrations, open('df_registrations.pickle','wb'))